In [3]:
import numpy as np

In [4]:
training_data = np.load(f'../fashion_train.npy')
test_data = np.load(f'../fashion_test.npy')

labels = training_data[:, -1]
training_data = training_data[:,:-1] / 255
training_data = np.c_[training_data, labels]

np.unique(training_data[:,-1])

array([0., 1., 2., 3., 4.])

In [5]:
import random
random.seed(42)

In [15]:
# Helper functions
def MSE(y_pred, y):
    return np.mean([(y_i - y_pred_i)**2 for y_i, y_pred_i in zip(y, y_pred)])

def MSE_derivative(y_pred, y):
    return [2 * (y_i - y_pred_i) for y_i, y_pred_i in zip(y, y_pred)]

def CrossEntropy(y_pred, y):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(np.sum(y * np.log(y_pred), axis=1))

def CrossEntropy_derivative(y_pred, y):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return y_pred - y

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def reLU(x):
    return np.maximum(0, x)

def reLU_derivative(x):
    return np.where(x > 0, 1, 0)

def leaky_reLU(x):
    return np.maximum(0.01 * x, x)

def leaky_reLU_derivative(x):
    return np.where(x > 0, 1, 0.01)

def ELU(x):
    return np.maximum(0.01 * (np.exp(x)-1), x)

def ELU_derivative(x):
    return np.where(x > 0, 1, 0.01* np.exp(x))

def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
    return probabilities

def xavier_initialization(input_size, output_size):
    return np.random.randn(input_size, output_size) * np.sqrt(1 / input_size)

def he_initialization(input_size, output_size):
    return np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)


def feedforward(data, num_hidden_layers, learning_rate, epochs, activation_function):

    # Select activation function
    if activation_function == 'sigmoid':
        initialization = xavier_initialization
        activation_function_ = sigmoid
        activation_function_derivative_ = sigmoid_derivative
        cost_function_ = MSE
        cost_derivative_ = MSE_derivative
    elif activation_function == 'relu':
        initialization = he_initialization
        activation_function_ = reLU
        activation_function_derivative_ = reLU_derivative
        cost_function_ = CrossEntropy
        cost_derivative_ = CrossEntropy_derivative
    elif activation_function == 'leaky_relu':
        initialization = he_initialization
        activation_function_ = leaky_reLU
        activation_function_derivative_ = leaky_reLU_derivative
        cost_function_ = CrossEntropy
        cost_derivative_ = CrossEntropy_derivative
    elif activation_function == 'ELU':
        initialization = he_initialization
        activation_function_ = ELU
        activation_function_derivative_ = ELU_derivative
        cost_function_ = CrossEntropy
        cost_derivative_ = CrossEntropy_derivative
    else:
        raise ValueError("Invalid activation function. Choose 'sigmoid', 'leaky_relu', 'ELU' or 'relu'.")
        
    # Weight initialization
    num_samples, num_features = data.shape[0], data.shape[1]-1
    input_layer_size = num_features
    hidden_layer_size = num_features // num_hidden_layers

    input_layer_weights = initialization(input_layer_size, hidden_layer_size)
    input_layer_bias = np.zeros((1, hidden_layer_size))

    hidden_layer_weights = [initialization(hidden_layer_size, hidden_layer_size) for _ in range(num_hidden_layers)]
    hidden_layer_bias = [np.zeros((1, hidden_layer_size)) for _ in range(num_hidden_layers)]

    output_layer_size = len(np.unique(data[:,-1]))
    output_layer_weights = initialization(hidden_layer_size, output_layer_size)
    output_layer_bias = np.zeros((1, output_layer_size))

    # -------------------------------------------------------------------------------------------------------------

    labels = data[:, -1]
    num_classes = len(np.unique(labels))
    classes = np.eye(num_classes)[labels.astype(int)]

    hidden_layer_inputs = [None] * num_hidden_layers
    hidden_layer_outputs = [None] * num_hidden_layers

    current_output = np.zeros((num_samples, output_layer_size))

    # first forward pass
    for i in range(num_hidden_layers):
        if i == 0:
            hidden_layer_inputs[i] = np.dot(data[:,:-1], input_layer_weights) + input_layer_bias
        else:
            hidden_layer_inputs[i] = np.dot(hidden_layer_outputs[i-1], hidden_layer_weights[i-1]) + hidden_layer_bias[i-1]

        hidden_layer_outputs[i] = activation_function_(hidden_layer_inputs[i])


    output_layer_input = np.dot(hidden_layer_outputs[-1], output_layer_weights) + output_layer_bias

    # Activation function for the output should be softmax becasue we have multiple classes
    output_layer_output = softmax(output_layer_input)
    current_output = output_layer_output
    # -------------------------------------------------------------------------------------------------------------

    # Backpropagation
    for epoch in range(epochs):

        # Compute cost
        cost = cost_function_(current_output, classes)
        
        # Weight and bias updates for the output layer
        delta = cost_derivative_(current_output, classes) * activation_function_derivative_(output_layer_input)
        gradient_output_layer_weights = np.dot(hidden_layer_outputs[-1].T, delta)
        gradient_output_layer_bias = np.sum(delta, axis=0, keepdims=True)

        output_layer_weights -= learning_rate * gradient_output_layer_weights
        output_layer_bias -= learning_rate * gradient_output_layer_bias

        # Weight and bias updates for hidden layers
        for i in range(num_hidden_layers-1, -1, -1):
            if i == num_hidden_layers-1:
                # For the last hidden layer
                delta = np.dot(delta, output_layer_weights.T) * activation_function_derivative_(hidden_layer_inputs[i])
            else:
                # For other hidden layers
                delta = np.dot(delta, hidden_layer_weights[i+1].T) * activation_function_derivative_(hidden_layer_inputs[i])
            
            # Compute gradients
            if i > 0:
                # For layers with a previous hidden layer
                gradient_hidden_layer_weights = np.dot(hidden_layer_outputs[i-1].T, delta)
            else:
                # For the first hidden layer (connected to input layer)
                gradient_hidden_layer_weights = np.dot(data[:,:-1].T, delta)
            
            gradient_hidden_layer_bias = np.sum(delta, axis=0, keepdims=True)
            
            # Update weights and biases
            if i > 0:
                hidden_layer_weights[i] -= learning_rate * gradient_hidden_layer_weights
                hidden_layer_bias[i] -= learning_rate * gradient_hidden_layer_bias
            else:
                input_layer_weights -= learning_rate * gradient_hidden_layer_weights
                input_layer_bias -= learning_rate * gradient_hidden_layer_bias

        # Forward pass
        for i in range(num_hidden_layers):
            if i == 0:
                hidden_layer_inputs[i] = np.dot(data[:,:-1], input_layer_weights) + input_layer_bias
            else:
                hidden_layer_inputs[i] = np.dot(hidden_layer_inputs[i-1], hidden_layer_weights[i-1]) + hidden_layer_bias[i-1]

            hidden_layer_outputs[i] = activation_function_(hidden_layer_inputs[i])

            output_layer_input = np.dot(hidden_layer_outputs[-1], output_layer_weights) + output_layer_bias
            output_layer_output = softmax(output_layer_input)
            
        current_output = output_layer_output

        # if epoch % 10 == 0:
        print(f'epoch: {epoch}, loss: {np.mean(cost)}')


    return current_output

tmp = feedforward(training_data, 2, 0.000005, 200, 'leaky_relu')

epoch: 0, loss: 1.6978854635849696
epoch: 1, loss: 1.759686982825285
epoch: 2, loss: 1.7130786893125773
epoch: 3, loss: 1.6654278383523982
epoch: 4, loss: 1.6572890669560092
epoch: 5, loss: 1.634593310154228
epoch: 6, loss: 1.6358336232516861
epoch: 7, loss: 1.6278669094735219
epoch: 8, loss: 1.6252554738903593
epoch: 9, loss: 1.621416312308237
epoch: 10, loss: 1.6185758006194435
epoch: 11, loss: 1.6150030687745551


KeyboardInterrupt: 

In [18]:
def forward_pass(data, weights, biases, activation_function):
    inputs, outputs = [], []
    current_input = data

    for w, b in zip(weights.values(), biases.values()):
        print("w_shape", w.shape)
        print("current_input_shape", current_input.shape)
        z = np.dot(current_input, w) + b
        print("z_shape", z.shape)
        inputs.append(z)
        current_input = activation_function(z)
        outputs.append(current_input)
    
    return inputs, outputs


def backward_pass(data, labels, weights, biases, inputs, outputs, activation_function_derivative, cost_derivative, learning_rate):
    print("output shape", outputs[-1].shape)
    print("labels shape", labels.shape)
    print("inputs shape", inputs[-1].shape)
    print(outputs)
    delta = cost_derivative(outputs[-1], labels) * activation_function_derivative(inputs[-1])
    
    for i in range(len(weights)):
        if weights[i].keys() != "hidden":
            delta = np.dot(delta, weights[i].T if i > 0 else data[:,:-1].T) * activation_function_derivative(inputs[i])
            gradient_weights = np.dot(outputs[i-1].T, delta)
            gradient_biases = np.sum(delta, axis=0, keepdims=True)

            weights[i] -= learning_rate * gradient_weights
            biases[i] -= learning_rate * gradient_biases
        else:
            for k in range(len(weights[i])):
                delta = np.dot(delta, weights[i][k].T) * activation_function_derivative(inputs[i][k])
                gradient_weights = np.dot(outputs[i-1].T, delta)
                gradient_biases = np.sum(delta, axis=0, keepdims=True)

                weights[i][k] -= learning_rate * gradient_weights
                biases[i][k] -= learning_rate * gradient_biases
   
    return weights, biases


In [19]:
# Helper functions
def MSE(y_pred, y):
    return 2 * np.mean([(y_i - y_pred_i)**2 for y_i, y_pred_i in zip(y, y_pred)])

def MSE_derivative(y_pred, y):
    return [(y_i - y_pred_i) for y_i, y_pred_i in zip(y, y_pred)]

def CrossEntropy(y_pred, y):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return -np.mean(np.sum(y * np.log(y_pred), axis=1))

def CrossEntropy_derivative(y_pred, y):
    epsilon = 1e-15
    y_pred = np.clip(y_pred, epsilon, 1 - epsilon)
    return y_pred - y

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def sigmoid_derivative(x):
    s = sigmoid(x)
    return s * (1 - s)

def reLU(x):
    return np.maximum(0, x)

def reLU_derivative(x):
    return np.where(x > 0, 1, 0)

def leaky_reLU(x):
    return np.maximum(0.01 * x, x)

def leaky_reLU_derivative(x):
    return np.where(x > 0, 1, 0.01)

def ELU(x):
    return np.maximum(0.01 * (np.exp(x)-1), x)

def ELU_derivative(x):
    return np.where(x > 0, 1, 0.01* np.exp(x))

def softmax(x):
    exp_values = np.exp(x - np.max(x, axis=1, keepdims=True))
    probabilities = exp_values / np.sum(exp_values, axis=1, keepdims=True)
    return probabilities

def xavier_initialization(input_size, output_size):
    return np.random.randn(input_size, output_size) * np.sqrt(1 / input_size)

def he_initialization(input_size, output_size):
    return np.random.randn(input_size, output_size) * np.sqrt(2 / input_size)


def feedforward2(data, num_hidden_layers, learning_rate, epochs, activation_function):

    # Select activation function
    if activation_function == 'sigmoid':
        initialization = xavier_initialization
        activation_function_ = sigmoid
        activation_function_derivative_ = sigmoid_derivative
        cost_function_ = MSE
        cost_derivative_ = MSE_derivative
    elif activation_function == 'relu':
        initialization = he_initialization
        activation_function_ = reLU
        activation_function_derivative_ = reLU_derivative
        cost_function_ = CrossEntropy
        cost_derivative_ = CrossEntropy_derivative
    elif activation_function == 'leaky_relu':
        initialization = he_initialization
        activation_function_ = leaky_reLU
        activation_function_derivative_ = leaky_reLU_derivative
        cost_function_ = CrossEntropy
        cost_derivative_ = CrossEntropy_derivative
    elif activation_function == 'ELU':
        initialization = he_initialization
        activation_function_ = ELU
        activation_function_derivative_ = ELU_derivative
        cost_function_ = CrossEntropy
        cost_derivative_ = CrossEntropy_derivative
    else:
        raise ValueError("Invalid activation function. Choose 'sigmoid', 'leaky_relu', 'ELU' or 'relu'.")
        
    # Weight initialization
    num_samples, num_features = data.shape[0], data.shape[1]-1
    input_layer_size = num_features
    hidden_layer_size = num_features // num_hidden_layers

    output_layer_size = len(np.unique(data[:,-1]))

    weights = {
        "input": initialization(input_layer_size, hidden_layer_size),
        "hidden": np.array([initialization(hidden_layer_size, hidden_layer_size) for _ in range(num_hidden_layers)]),
        "output": initialization(hidden_layer_size, output_layer_size)
    }
    biases = {
        "input": np.zeros((1, hidden_layer_size)),
        "hidden": np.zeros((1, hidden_layer_size)),
        "output": np.zeros((1, output_layer_size))
    }

    # -------------------------------------------------------------------------------------------------------------

    labels = data[:, -1]
    num_classes = len(np.unique(labels))
    classes = np.eye(num_classes)[labels.astype(int)]
    input, output = forward_pass(data[:, :-1], weights, biases, activation_function_)
  
    weights, biases = backward_pass(data, classes, weights, biases, input, output, activation_function_derivative_, cost_derivative_, learning_rate)

    print(weights)
    return weights

feedforward2(training_data, 2, 0.000001, 3, 'relu')

w_shape (784, 392)
current_input_shape (10000, 784)
z_shape (10000, 392)
w_shape (2, 392, 392)
current_input_shape (10000, 392)
z_shape (10000, 2, 392)
w_shape (392, 5)
current_input_shape (10000, 2, 392)
z_shape (10000, 2, 5)
output shape (10000, 2, 5)
labels shape (10000, 5)
inputs shape (10000, 2, 5)
[array([[0.22539573, 0.        , 0.95737522, ..., 0.38700935, 0.        ,
        0.41582177],
       [0.13308202, 0.26426893, 0.50084099, ..., 0.2174232 , 0.        ,
        0.17742599],
       [0.44477333, 0.        , 0.92245097, ..., 0.52439474, 0.        ,
        0.49124787],
       ...,
       [0.        , 0.22537091, 0.6318817 , ..., 0.56941705, 0.        ,
        0.88111488],
       [0.52715771, 0.        , 0.51151227, ..., 0.35362733, 0.        ,
        0.51339508],
       [0.        , 0.        , 0.54485478, ..., 0.40421756, 0.        ,
        0.5240811 ]]), array([[[0.        , 0.92491724, 0.62100167, ..., 0.        ,
         0.        , 0.        ],
        [0.        ,

ValueError: operands could not be broadcast together with shapes (10000,2,5) (10000,5) 